In [2]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add(url = "https://github.com/SimonEnsemble/Xtals.jl.git")

  Resolving package versions...
No Changes to `C:\Users\kgeri\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\kgeri\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...


No Changes to `C:\Users\kgeri\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\kgeri\.julia\environments\v1.5\Manifest.toml`
   Updating git-repo `https://github.com/SimonEnsemble/Xtals.jl.git`


  Resolving package versions...


LoadError: Unsatisfiable requirements detected for package DataFrames [a93c6f00]:
 DataFrames [a93c6f00] log:
 ├─possible versions are: [0.11.7, 0.12.0, 0.13.0-0.13.1, 0.14.0-0.14.1, 0.15.0-0.15.2, 0.16.0, 0.17.0-0.17.1, 0.18.0-0.18.4, 0.19.0-0.19.4, 0.20.0-0.20.2, 0.21.0-0.21.8, 0.22.0-0.22.2] or uninstalled
 ├─restricted to versions 0.22.1-0.22 by Xtals [ede5f01d], leaving only versions 0.22.1-0.22.2
 │ └─Xtals [ede5f01d] log:
 │   ├─possible versions are: 0.2.0 or uninstalled
 │   └─Xtals [ede5f01d] is fixed to version 0.2.0
 └─restricted to versions 0.20 by PorousMaterials [68953c7c] — no versions left
   └─PorousMaterials [68953c7c] log:
     ├─possible versions are: 0.2.0 or uninstalled
     └─PorousMaterials [68953c7c] is fixed to version 0.2.0

In [1]:
using DataFrames
using CSV
using Printf
using Statistics
using LinearAlgebra

In [2]:
using Xtals
using LightGraphs
using GraphPlot

In [113]:
mol_file = "1.mol"

"1.mol"

In [114]:
atoms, bonds, bondtypes = read_mol(joinpath(homedir(), "Documents", "GitHub", "MolecularSynthesis", "examples", mol_file))
#atoms = read_xyz(joinpath(homedir(), "Documents", "Grad School", "Research", "examples", "1.xyz"))

(Atoms{Cart}(31, [:O, :C, :O, :C, :H, :C, :C, :C, :C, :C  …  :H, :H, :N, :C, :C, :H, :H, :O, :O, :H], Cart([1.428 0.344 … 1.716 0.039; 0.001 0.423 … 12.315 13.786; 0.64 -0.048 … -0.421 -0.14])), {31, 32} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0), [1, 1, 2, 1, 5, 5, 5, 1, 1, 5  …  1, 2, 1, 1, 2, 1, 1, 2, 1, 1])

In [115]:
r = maximum([distance(atoms, i, j) for i = 1:atoms.n, j = 1:atoms.n])
box_length = 3*r

42.4262013147536

In [116]:
box = Box(box_length,box_length,box_length)
#box = Box(30.0, 30.0, 30.0)
atoms=Frac(atoms, box)

Atoms{Frac}(31, [:O, :C, :O, :C, :H, :C, :C, :C, :C, :C  …  :H, :H, :N, :C, :C, :H, :H, :O, :O, :H], Frac([0.047599999999999996 0.011466666666666665 … 0.05719999999999997 0.001299999999999972; 3.333333333333203e-5 0.0141 … 0.4105 0.4595333333333333; 0.021333333333333333 -0.0016 … -0.014033333333333333 -0.004666666666666667]))

In [117]:
crystal = Crystal("NewTest.cif", box, atoms, Charges{Frac}(0), bonds, Xtals.SymmetryInfo())

Name: NewTest.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 30.000000 Å. b = 30.000000 Å, c = 30.000000 Å
	Volume of unit cell: 27000.000000 Å³

	# atoms = 31
	# charges = 0
	chemical formula: Dict(:N => 4,:H => 9,:Br => 1,:O => 4,:C => 13)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [118]:
for ed in edges(crystal.bonds)
   
    println(ed)
    i = ed.src
    j = ed.dst
       
    species_i = crystal.atoms.species[i]
    species_j = crystal.atoms.species[j]
    println("atom ", species_i, " is connected to ", species_j)
    

end

Edge 1 => 2
atom O is connected to C
Edge 1 => 5
atom O is connected to H
Edge 2 => 3
atom C is connected to O
Edge 2 => 4
atom C is connected to C
Edge 4 => 6
atom C is connected to C
Edge 4 => 10
atom C is connected to C
Edge 6 => 7
atom C is connected to C
Edge 6 => 13
atom C is connected to Br
Edge 7 => 8
atom C is connected to C
Edge 7 => 12
atom C is connected to C
Edge 8 => 9
atom C is connected to C
Edge 8 => 11
atom C is connected to C
Edge 9 => 10
atom C is connected to C
Edge 9 => 15
atom C is connected to H
Edge 10 => 14
atom C is connected to H
Edge 11 => 16
atom C is connected to C
Edge 11 => 19
atom C is connected to N
Edge 11 => 23
atom C is connected to H
Edge 12 => 24
atom C is connected to N
Edge 16 => 17
atom C is connected to N
Edge 16 => 21
atom C is connected to H
Edge 16 => 22
atom C is connected to H
Edge 17 => 18
atom N is connected to N
Edge 17 => 20
atom N is connected to C
Edge 18 => 19
atom N is connected to N
Edge 20 => 25
atom C is connected to C
Edge 20

In [119]:
### Function to identify carboxyl. When species is carbon, checks if it has two oxygen neighbors. 
### If it does, returns oxygen id's, anchor id, and oxygen's hydrogen id

function identify_carboxyl(crystal::Crystal, a::Int64)
   
    species = crystal.atoms.species[a]    #Identify species of selected atom
    nbs = neighbors(crystal.bonds, a)     #Identify neighbors of atom
    
    if species == :C                      #Identifying carboxylate starts with carbon
        oxygen_counter = 0
        oxygen_id = Int64[]
        X_id = 0
        H_id = 0
        for nb in nbs                     #Iterating through neighbors via identifying number
            species_nb = crystal.atoms.species[nb] #Get species of neighbor
            if species_nb == :O
                
                next_nbs = neighbors(crystal.bonds, nb)  #Get neighbors of oxygen (finding hydrogen)
                
                for next_nb in next_nbs
                    species_next_nb = crystal.atoms.species[next_nb]  #Get species of oxygen's neighbors
                    if species_next_nb == :H     #Identify hydrogen neighbor
                        H_id = next_nb
                    else
                    end
                end
                
                oxygen_counter += 1
                push!(oxygen_id, nb)         #Enter oxygen id into array
            else
                X_id = nb            #If neighbor of carboxylate carbon isn't oxygen, must be anchor
            end
        end
        
        if oxygen_counter == 2   #Assuming carboxylate carbon is only carbon with two oxygens bonded
        
            return true, oxygen_id, X_id, H_id
        else 
            return false, [], 0, 0
        end
    else
        return false, [], 0, 0
    end

end

identify_carboxyl (generic function with 1 method)

In [120]:
identify_carboxyl(crystal, 26)

(true, [29, 30], 25, 31)

In [121]:
#

keep = [true for i = 1:crystal.atoms.n]

X_species = Symbol[]
X_ids = Int64[]

for a = 1:crystal.atoms.n
       
   

    is_carboxyl, oxygen_ids, X_id, H_id = identify_carboxyl(crystal, a)
    
    if is_carboxyl == true 
    
        push!(X_species, crystal.atoms.species[X_id])
        push!(X_ids, X_id)
        crystal.atoms.species[X_id] = :X
        @assert length(oxygen_ids) == 2
        keep[oxygen_ids] .= false
        keep[a] = false
        keep[H_id] = false
        
    end


end

In [122]:
X_species
#X_ids

2-element Array{Symbol,1}:
 :C
 :C

In [123]:
for i in [19, 4]
    
   h = findfirst(isequal(i), X_ids) 
    print(h, "\n")
end

nothing
1


In [124]:
h = findfirst(isequal(19), X_ids) 
#X_species[h]

In [125]:
for ed in edges(crystal.bonds)
   
    println(ed)
    i = ed.src
    j = ed.dst
       
    species_i = crystal.atoms.species[i]
    species_j = crystal.atoms.species[j]
    println("atom ", species_i, " is connected to ", species_j)
    

end

Edge 1 => 2
atom O is connected to C
Edge 1 => 5
atom O is connected to H
Edge 2 => 3
atom C is connected to O
Edge 2 => 4
atom C is connected to X
Edge 4 => 6
atom X is connected to C
Edge 4 => 10
atom X is connected to C
Edge 6 => 7
atom C is connected to C
Edge 6 => 13
atom C is connected to Br
Edge 7 => 8
atom C is connected to C
Edge 7 => 12
atom C is connected to C
Edge 8 => 9
atom C is connected to C
Edge 8 => 11
atom C is connected to C
Edge 9 => 10
atom C is connected to C
Edge 9 => 15
atom C is connected to H
Edge 10 => 14
atom C is connected to H
Edge 11 => 16
atom C is connected to C
Edge 11 => 19
atom C is connected to N
Edge 11 => 23
atom C is connected to H
Edge 12 => 24
atom C is connected to N
Edge 16 => 17
atom C is connected to N
Edge 16 => 21
atom C is connected to H
Edge 16 => 22
atom C is connected to H
Edge 17 => 18
atom N is connected to N
Edge 17 => 20
atom N is connected to C
Edge 18 => 19
atom N is connected to N
Edge 20 => 25
atom C is connected to X
Edge 20

In [126]:
crystal.atoms.n

31

In [127]:
#Make a boolean array to keep atoms. 
crystal.atoms[keep]

Atoms{Frac}(23, [:X, :C, :C, :C, :C, :C, :C, :C, :Br, :H  …  :N, :N, :C, :H, :H, :H, :N, :X, :H, :H], Frac([0.005399999999999997 -0.03433333333333334 … -0.03390000000000002 0.06083333333333331; 0.06343333333333333 0.08266666666666667 … 0.3533666666666667 0.32133333333333336; -0.0008333333333333334 0.008666666666666666 … 0.0107 -0.013433333333333334]))

In [128]:
write_cif(crystal, "tobacco_crystal.cif")

In [129]:
crystal

Name: NewTest.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 30.000000 Å. b = 30.000000 Å, c = 30.000000 Å
	Volume of unit cell: 27000.000000 Å³

	# atoms = 31
	# charges = 0
	chemical formula: Dict(:N => 4,:H => 9,:X => 2,:Br => 1,:O => 4,:C => 11)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [130]:
write_cif(crystal, "tobacco_crystal.cif")

In [131]:
#infer_bonds!(tobacco_crystal, true)
#remove_bonds!(tobacco_crystal)

In [132]:
tobacco_crystal = crystal[BitArray(keep)] #Crystal("tobacco_crystal_2.cif", box, atoms[keep], Charges{Frac}(0)) #bonds, Xtals.SymmetryInfo())

Name: NewTest.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 30.000000 Å. b = 30.000000 Å, c = 30.000000 Å
	Volume of unit cell: 27000.000000 Å³

	# atoms = 23
	# charges = 0
	chemical formula: Dict(:N => 4,:H => 7,:X => 2,:Br => 1,:C => 9)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [133]:
function center_mass(crystal::Crystal)
    xf_cm = sum(crystal.atoms.coords.xf, dims = 2)
    frctn_xf_cm = xf_cm ./ crystal.atoms.n
    new_coords = crystal.atoms.coords.xf .- frctn_xf_cm
    crystal.atoms.coords.xf .= mod.(crystal.atoms.coords.xf, 1)
    crystal.atoms.coords.xf .= new_coords
    
    return crystal
end

center_mass (generic function with 1 method)

In [134]:
tobacco_crystal = center_mass(tobacco_crystal)

Name: NewTest.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 30.000000 Å. b = 30.000000 Å, c = 30.000000 Å
	Volume of unit cell: 27000.000000 Å³

	# atoms = 23
	# charges = 0
	chemical formula: Dict(:N => 4,:H => 7,:X => 2,:Br => 1,:C => 9)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [135]:
tobacco_crystal.atoms

Atoms{Frac}(23, [:X, :C, :C, :C, :C, :C, :C, :C, :Br, :H  …  :N, :N, :C, :H, :H, :H, :N, :X, :H, :H], Frac([0.006195652173913051 -0.03353768115942029 … -0.03310434782608697 0.06162898550724636; -0.1305 -0.11126666666666668 … 0.15943333333333337 0.1274; 0.002323188405797102 0.011823188405797103 … 0.013856521739130434 -0.010276811594202899]))

In [136]:
"""
    write_cif(crystal, filename; fractional_coords=true, number_atoms=true)
Write a `crystal::Crystal` to a .cif file with `filename::AbstractString`. If `filename` does
not include the .cif extension, it will automatically be added. the `fractional_coords` flag
allows us to write either fractional or Cartesian coordinates.
"""
function write_cif_Kai(crystal::Crystal, filename::AbstractString, X_species::Array{Symbol,1}, X_ids::Array{Int64,1};
		 fractional_coords::Bool=true, number_atoms::Bool=true)
    if has_charges(crystal)
        if crystal.atoms.n != crystal.charges.n
            error("write_cif assumes equal numbers of Charges and Atoms (or zero charges)")
        end
        if ! isapprox(crystal.charges.coords, crystal.atoms.coords)
            error("write_cif needs coords of atoms and charges to correspond.")
        end
    end

    # TODO is this labeling necessary for the bonds, arthur?
    # create dictionary for tracking label numbers
    label_numbers = Dict{Symbol, Int}()
    for atom in crystal.atoms.species
        if !haskey(label_numbers, atom)
            label_numbers[atom] = 1
        end
    end

    # append ".cif" to filename if it doesn't already have the extension
    if ! occursin(".cif", filename)
        filename *= ".cif"
    end
    cif_file = open(filename, "w")
    # first line should be data_xtalname_PM
    if crystal.name == ""
        @printf(cif_file, "data_PM\n")
    else
        # don't include file extension!
        @printf(cif_file, "data_%s_PM\n", split(crystal.name, ".")[1])
    end

    @printf(cif_file, "_symmetry_space_group_name_H-M\t'%s'\n", crystal.symmetry.space_group)

    @printf(cif_file, "_cell_length_a\t%f\n", crystal.box.a)
    @printf(cif_file, "_cell_length_b\t%f\n", crystal.box.b)
    @printf(cif_file, "_cell_length_c\t%f\n", crystal.box.c)

    @printf(cif_file, "_cell_angle_alpha\t%f\n", crystal.box.α * 180.0 / pi)
    @printf(cif_file, "_cell_angle_beta\t%f\n", crystal.box.β * 180.0 / pi)
    @printf(cif_file, "_cell_angle_gamma\t%f\n", crystal.box.γ * 180.0 / pi)

    @printf(cif_file, "_symmetry_Int_Tables_number 1\n\n")
    @printf(cif_file, "loop_\n_symmetry_equiv_pos_as_xyz\n")
    for i in 1:size(crystal.symmetry.operations, 2)
        @printf(cif_file, "'%s,%s,%s'\n", crystal.symmetry.operations[:, i]...)
    end
    @printf(cif_file, "\n")

    @printf(cif_file, "loop_\n_atom_site_label\n_atom_site_type_symbol\n")
    if fractional_coords
        @printf(cif_file, "_atom_site_fract_x\n_atom_site_fract_y\n_atom_site_fract_z\n")
    else
        @printf(cif_file, "_atom_site_Cartn_x\n_atom_site_Cartn_y\n_atom_site_Cartn_z\n")
    end
    high_precision_charges = false # if, for neutrality, need high-precision charges
    if has_charges(crystal)
        @printf(cif_file, "_atom_site_charge\n")
        # if crystal will not be charge neutral to a 1e-5 tolerance when loading it
        #    into PorousMaterials.jl, then write higher-precision charges
        if abs(sum(round.(crystal.charges.q, digits=6))) > NET_CHARGE_TOL
            @info "writing high-precision charges for " * filename * ".\n"
            high_precision_charges = true
        end
    end

    idx_to_label = Array{AbstractString, 1}(undef, crystal.atoms.n)
    nb_of_X = 0
    for i = 1:crystal.atoms.n
        # print label and type symbol
        
        if crystal.atoms.species[i] == :X
            nb_of_X += 1
            #X_index = findfirst(isequal(i), X_ids)
            #println(X_index)
            element = X_species[nb_of_X]
            
            @printf(cif_file, "%s\t%s\t", string(crystal.atoms.species[i]) *
                (number_atoms ? string(label_numbers[crystal.atoms.species[i]]) : ""),
                element)
        else
            @printf(cif_file, "%s\t%s\t", string(crystal.atoms.species[i]) *
                (number_atoms ? string(label_numbers[crystal.atoms.species[i]]) : ""),
                crystal.atoms.species[i])
        end
        
        # store label for this atom idx
        idx_to_label[i] = string(crystal.atoms.species[i]) *
                    string(label_numbers[crystal.atoms.species[i]])
        # increment label
        label_numbers[crystal.atoms.species[i]] += 1
        if fractional_coords
            @printf(cif_file, "%f\t%f\t%f", crystal.atoms.coords.xf[:, i]...)
        else
            @printf(cif_file, "%f\t%f\t%f", (crystal.box.f_to_c * crystal.atoms.coords.xf[:, i])...)
        end
        if has_charges(crystal)
            if high_precision_charges
                @printf(cif_file, "\t%.10f\n", crystal.charges.q[i])
            else
                @printf(cif_file, "\t%f\n", crystal.charges.q[i])
            end
        else
            @printf(cif_file, "\n")
        end
    end

    # only print bond information if it is in the crystal
    if ne(crystal.bonds) > 0
        if ! number_atoms
             error("must label atoms with numbers to write bond information.\n")
        end
        # print column names for bond information
        @printf(cif_file, "\nloop_\n_geom_bond_atom_site_label_1\n_geom_bond_atom_site_label_2\n_geom_bond_distance\n_ccdc_geom_bond_type\n")

        for edge in collect(edges(crystal.bonds))
            dxf = crystal.atoms.coords.xf[:, edge.src] - crystal.atoms.coords.xf[:, edge.dst]
            nearest_image!(dxf)
            @printf(cif_file, "%s\t%s\t%0.5f\t%s\n", idx_to_label[edge.src], idx_to_label[edge.dst],
                    norm(dxf), ". A")
        end
    end
    close(cif_file)
end

write_cif_Kai

In [137]:
X_species

2-element Array{Symbol,1}:
 :C
 :C

In [138]:
X_ids

2-element Array{Int64,1}:
  4
 25

In [139]:
infer_bonds!(tobacco_crystal, false)

┌ Warning: atom 1 = X in NewTest.cif is not bonded to any other atom.
└ @ Xtals C:\Users\kgeri\.julia\packages\Xtals\Hr2Rt\src\bonds.jl:430


false

In [140]:
write_cif_Kai(tobacco_crystal, "tobacco_crystal_2.cif", X_species, X_ids)

In [141]:
mol_file = replace(mol_file, "mol" => "cif")
test_file_name = string("Test", mol_file)

"Test1.cif"

In [142]:
write_cif_Kai(tobacco_crystal, test_file_name, X_species, X_ids)